In [1]:
import numpy as np
import pandas as pd
from sklearn.cross_validation import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn import linear_model
from scipy.stats import *

from ipywidgets import interact, interactive, fixed, widgets

from bokeh.plotting import figure
from bokeh.io import output_notebook, show#, output_file #Descomentar para guardar el archivo .html
from bokeh.models.widgets import Panel, Tabs
output_notebook()

Loading BokehJS ...

In [2]:
df = pd.read_csv("regLin.csv")

In [3]:
def sets(data_frame):
    entrenamiento, prueba = train_test_split(data_frame, test_size=0.2)#, random_state=18)
    return entrenamiento, prueba

def como_vectores(df):
    entrenamiento, prueba = sets(df)
    entrenamiento_y = entrenamiento.y.as_matrix()
    entrenamiento_x = entrenamiento.X.as_matrix()
    prueba_y = prueba.y.as_matrix()
    prueba_x = prueba.X.as_matrix()
    return entrenamiento_x, entrenamiento_y, prueba_x, prueba_y

def normalizar(df):
    entrenamiento_x, entrenamiento_y, prueba_x, prueba_y = como_vectores(df)
    entrenamiento_y_n = entrenamiento_y/max(entrenamiento_y)
    entrenamiento_x_n = entrenamiento_x/max(entrenamiento_x)
    prueba_y_n = prueba_y/max(df.y)
    prueba_x_n = prueba_x/max(df.X)
    return entrenamiento_x_n, entrenamiento_y_n, prueba_x_n, prueba_y_n

def agregar_unos(df,norm):
    if norm == False:
        entrenamiento_x, entrenamiento_y, prueba_x, prueba_y = como_vectores(df)
        unos = np.ones(len(entrenamiento_x))
        matriz = np.dstack((unos,entrenamiento_x))
        matriz = matriz[0,:]
        return matriz, entrenamiento_x, entrenamiento_y, prueba_x, prueba_y
    elif norm == True:
        entrenamiento_x, entrenamiento_y, prueba_x, prueba_y = normalizar(df)
        unos = np.ones(len(entrenamiento_x))
        matriz = np.dstack((unos,entrenamiento_x))
        matriz = matriz[0]
        return matriz, entrenamiento_x, entrenamiento_y, prueba_x, prueba_y

In [4]:
def LMS(df,eta,norm):
    """
    Función de eta
    """
    matriz, entrenamiento_x, entrenamiento_y, prueba_x, prueba_y = agregar_unos(df,norm)
    
    w = np.array([1.,1.])

    error = []
    W_0 = []
    W_1 = []

    contador = 0

    while (sum([np.dot(w,matriz[i]) - entrenamiento_y[i] for i in range(0,len(entrenamiento_y))]))**2 > 0.01:
        error.append((sum([np.dot(w,matriz[i]) - entrenamiento_y[i] for i in range(0,len(entrenamiento_y))]))**2)
        W_0.append(w[0])
        W_1.append(w[1])
        w[0] = w[0] + (eta * (sum([(entrenamiento_y[i] - np.dot(w,matriz[i])) * matriz[i,0] \
                                   for i in range(0,len(entrenamiento_y))])/len(entrenamiento_y)))
        w[1] = w[1] + (eta * (sum([(entrenamiento_y[i] - np.dot(w,matriz[i])) * matriz[i,1] \
                                   for i in range(0,len(entrenamiento_y))])/len(entrenamiento_y)))
        contador += 1
        
    return error, w, W_0, W_1, contador, matriz, entrenamiento_x, entrenamiento_y, prueba_x, prueba_y

In [12]:
def graficar(eta,cota,norm):
        
    error, w, W_0, W_1, contador, matriz, entrenamiento_x, entrenamiento_y, prueba_x, prueba_y = LMS(df,eta,norm)
    
    LMS(df,eta,norm)
    
    s1 = figure(width=750, plot_height=500, title=None)
    s1.circle(entrenamiento_x,entrenamiento_y, size=5, color="red", alpha=0.5)
    s1.line(np.sort(entrenamiento_x), [w[1]*(np.sort(entrenamiento_x))[i]+w[0] for i in range(0,len(entrenamiento_x))],\
            line_width=1.5, color="navy", alpha=0.7)
    s1.title.text = "Regresión de entrenamiento"
    s1.xaxis.axis_label = "X"
    s1.yaxis.axis_label = "y"
    tab1 = Panel(child=s1, title="Entrenamiento")

    s2 = figure(width=750, plot_height=500, title=None)
    s2.circle(prueba_x,prueba_y, size=5, color="red", alpha=0.5)
    s2.line(np.sort(prueba_x), [w[1]*(np.sort(prueba_x))[i]+w[0] for i in range(0,len(prueba_x))], line_width=1.5,\
            color="navy", alpha=0.7)
    s2.title.text = "Regresión con la prueba"
    s2.xaxis.axis_label = "X"
    s2.yaxis.axis_label = "y"
    tab2 = Panel(child=s2, title="Prueba")

    s3 = figure(width=750, plot_height=500, title=None)
    s3.circle(W_0,error, size=5, color="red", alpha=0.5)
    s3.title.text = "Errores"
    s3.xaxis.axis_label = "X"
    s3.yaxis.axis_label = "y"
    tab3 = Panel(child=s3, title="Errores w0")

    s4 = figure(width=750, plot_height=500, title=None)
    s4.circle(W_1[cota:len(W_1)],error[cota:len(W_1)], size=5, color="red", alpha=0.5)
    s4.title.text = "Errores"
    s4.xaxis.axis_label = "X"
    s4.yaxis.axis_label = "y"
    tab4 = Panel(child=s4, title="Errores w1")
    
    s5 = figure(width=750, plot_height=500, title=None)
    s5.circle(range(cota,contador), error[cota:len(error)], size=5, color="red", alpha=0.5)
    s5.title.text = "Errores vs iteración"
    s5.xaxis.axis_label = "Iteración"
    s5.yaxis.axis_label = "Errores"
    tab5 = Panel(child=s5, title="Errores vs iteración")

    tabs = Tabs(tabs=[ tab1, tab2, tab3, tab4, tab5])
    
    show(tabs)
    
    print('Requirió {} iteraciones').format(contador)

interact(graficar, eta=widgets.BoundedFloatText(min=0.05,max=1,step=0.01,value=0.05, description='$\mu$: '),\
                   cota=widgets.IntSlider(min=0,max=100,step=1,value=0,description='Cota: '), \
                   norm=widgets.Checkbox(description='Normalizado',value=False))

Requirió 1290 iteraciones
